<a href="https://colab.research.google.com/github/pop-555/git_tutorial/blob/master/RAG/chapter04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. LangChain の基礎


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 4.1. LangChain の概要


### LangChain のインストール


#### 【注意】既知のエラーについて

pydantic のアップデートにより、明示的に pydantic のバージョンを指定していない箇所で ChatOpenAI などを使用すると、`PydanticUserError: 'ChatOpenAI' is not fully defined; you should define 'BaseCache', then call 'ChatOpenAI.model_rebuild()'.` というエラーが発生するようになりました。

このエラーは、`!pip install pydantic==2.10.6` のように、pydantic の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install pydantic==2.10.6` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 pydantic==2.10.6

### LangSmith のセットアップ


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # LangChainで「会話の動き（トレース）」を記録する機能をオンにするよ

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"  # LangChainのサーバーの場所（どこにアクセスするか）を教えてるよ

os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")  # 自分の秘密のカギ（APIキー）をColabから取り出して設定してるよ

os.environ["LANGCHAIN_PROJECT"] = "agent-book"  # プロジェクトの名前を「agent-book」にして、記録をまとめるよ


## 4.2. LLM / Chat model


### LLM


In [ ]:
from langchain_openai import OpenAI  # OpenAIを使うためのLangChainの道具を読み込むよ

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)  # GPTの種類を指定してAIを準備するよ（temperature=0は「ブレずに答えてね」って意味）

ai_message = model.invoke("こんにちは")  # AIに「こんにちは」って話しかけて返事をもらうよ

print(ai_message)  # 返ってきたAIの返事を画面に表示するよ




こんにちは

こんにちは、私はAIのアシスタントです。あなたのお手伝いをすることができます。何かお困りのことはありますか？


### Chat model


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage  # 人の発言やAIの発言を使えるようにするよ
from langchain_openai import ChatOpenAI  # ChatGPTを使う道具を読み込むよ

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # GPT-4のちょっと軽いバージョンで、まじめに答えるように設定するよ

messages = [  # これまでの会話の流れをまとめてるよ
    SystemMessage("You are a helpful assistant."),  # 「あなたは親切なAIです」という設定を伝えてるよ
    HumanMessage("こんにちは！私はジョンと言います"),  # 人（ジョンさん）があいさつしてるよ
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),  # AIが返事をしてるよ
    HumanMessage(content="私の名前がわかりますか？"),  # 人がもう一度質問してるよ
]

ai_message = model.invoke(messages)  # この会話の続きとしてAIに考えてもらうよ

print(ai_message.content)  # AIが返してきた答えを画面に出すよ


はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？


### ストリーミング


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage  # AIへのルール（System）や人の発言（Human）を作る道具を読み込むよ
from langchain_openai import ChatOpenAI  # ChatGPTを使うためのLangChainの道具を使うよ

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # まじめに返事する小型モデルを使うよ（温度0＝ブレが少ない）

messages = [  # 会話の内容をまとめておくよ
    SystemMessage("You are a helpful assistant."),  # AIには「親切なアシスタントでいてね」と伝えてるよ
    HumanMessage("こんにちは！"),  # 人が「こんにちは！」って話しかけてるよ
]

for chunk in model.stream(messages):  # AIから返事が少しずつ流れてくるのを受け取るよ
    print(chunk.content, end="", flush=True)  # 一文字ずつリアルタイムで表示するよ（改行なしで即表示！）


こんにちは！どのようにお手伝いできますか？

## 4.3. Prompt template


### PromptTemplate


In [ ]:
from langchain_core.prompts import PromptTemplate  # LangChainの「質問ひな形」を使うための道具をよび出すよ

prompt = PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名: {dish}""")  # 「料理名を入れたらレシピを考える」というテンプレートを作ってるよ

prompt_value = prompt.invoke({"dish": "カレー"})  # 「カレー」を入れて、テンプレートを完成させるよ

print(prompt_value.text)  # 出来上がった文（AIに渡す質問）を表示してるよ


以下の料理のレシピを考えてください。

料理名: カレー


#### ＜補足：プロンプトの変数が 1 つの場合＞


In [ ]:
prompt_value = prompt.invoke("カレー")  # ❌これは間違い！「カレー」だけを渡すとテンプレートがうまく動かないよ
print(prompt_value.text)  # 結果を表示しようとしてるけど、うまくいかないかもしれないよ


以下の料理のレシピを考えてください。

料理名: カレー


### ChatPromptTemplate


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 会話用のプロンプトテンプレートを使うために必要な道具を読み込むよ

prompt = ChatPromptTemplate.from_messages(  # 会話の流れを作るテンプレートを作るよ
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),  # 最初にAIへルール説明（"あなたはレシピを考える人です"）
        ("human", "{dish}"),  # ユーザーが料理名を言う部分だよ（{dish}が後で「カレー」になる）
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})  # {dish} に「カレー」を入れて、実際のプロンプトを完成させるよ
print(prompt_value)  # 出来上がった会話の流れを表示するよ（AIに送る内容がどうなるか確認できる）


messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage  # 人間とAIのセリフのためのクラスを読み込むよ
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 会話テンプレートと履歴の場所（プレースホルダー）を使うよ

prompt = ChatPromptTemplate.from_messages(  # 会話の形を作るテンプレートを作るよ
    [
        ("system", "You are a helpful assistant."),  # 最初にAIのルールを伝える（親切なアシスタントだよ）
        MessagesPlaceholder("chat_history", optional=True),  # これまでの会話（履歴）を入れる場所だよ
        ("human", "{input}"),  # 今の質問（ユーザーの発言）を入れる場所だよ
    ]
)

prompt_value = prompt.invoke(  # 実際の会話の内容を入れて、AIに渡す会話全体を作るよ
    {
        "chat_history": [  # 会話の履歴（前にどんな話をしたか）を入れるよ
            HumanMessage(content="こんにちは！私はジョンと言います！"),  # 人間が自己紹介したよ
            AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),  # AIが返事をしたよ
        ],
        "input": "私の名前が分かりますか？",  # 今の質問（ユーザーの新しい発言）を入れるよ
    }
)
print(prompt_value)  # 最終的にAIに渡す文章（すべての会話）を表示するよ


messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='こんにちは！私はジョンと言います！', additional_kwargs={}, response_metadata={}), AIMessage(content='こんにちは、ジョンさん！どのようにお手伝いできますか？', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前が分かりますか？', additional_kwargs={}, response_metadata={})]


### LangSmith の Prompts


In [ ]:
from langsmith import Client  # LangSmithという道具を使うよ。これはテンプレートを管理する場所だよ

client = Client()  # LangSmithとつながる準備をするよ（お客さん用の入口を作ってるイメージ）

prompt = client.pull_prompt("oshima/recipe")  # 「oshima」さんが作った「recipe」というテンプレートを引っ張ってくるよ

prompt_value = prompt.invoke({"dish": "カレー"})  # 「料理名＝カレー」としてテンプレートを使ってみるよ

print(prompt_value)  # 出てきたレシピを表示するよ


messages=[SystemMessage(content='ユーザーが入力した料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


### （コラム）マルチモーダルモデルの入力の扱い


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 会話のテンプレートを作るための道具を読み込むよ
from langchain_openai import ChatOpenAI  # OpenAIのAI（会話型）を使うための準備だよ

# 会話のテンプレートを作るよ（ユーザーの発言として、画像と文字を入れるよ）
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",  # ユーザーの発言として
            [
                {"type": "text", "text": "画像を説明してください。"},  # 「この画像を説明してね」と言ってるよ
                {"type": "image_url", "image_url": {"url": "{image_url}"}},  # あとで画像のURLをここに入れるよ
            ],
        ),
    ]
)

# 説明してもらいたい画像のURLを用意するよ（LangChain本の表紙画像）
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

# 画像のURLをテンプレートに当てはめて、AIに渡す準備をするよ
prompt_value = prompt.invoke({"image_url": image_url})


In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0)  # すごくかしこいAI（GPT-4o）を使う準備をしてるよ。温度0はブレずにしっかり考えてくれる設定だよ

ai_message = model.invoke(prompt_value)  # さっき作った「画像を説明してね」っていうお願いをAIに送って、返事をもらうよ

print(ai_message.content)  # AIからの返事（画像の説明）を画面に出すよ


この画像は、本の表紙です。タイトルは「ChatGPT/ LangChainによるチャットシステム構築[実践]入門」で、著者は吉田真吾と大嶋勇樹です。表紙にはカラフルな鳥のイラストが描かれています。内容としては、大規模言語モデルを本番システムで活用するための基礎知識と実践的なハンズオンが含まれているようです。OpenAI APIやLangChainの活用についても触れられています。


## 4.4. Output parser


### PydanticOutputParser を使った Python オブジェクトへの変換


In [ ]:
from pydantic import BaseModel, Field  # データの形を決めるための道具（型チェックもしてくれる）を使うよ

class Recipe(BaseModel):  # レシピという「料理の説明書」の型を作るよ
    ingredients: list[str] = Field(description="ingredients of the dish")  # 材料を文字のリストで入れるよ（例: ["にんじん", "たまねぎ"]）
    steps: list[str] = Field(description="steps to make the dish")  # 作り方も文字のリストで入れるよ（例: ["切る", "焼く"]）


In [ ]:
from langchain_core.output_parsers import PydanticOutputParser  # AIの答えをきれいに読み取る道具を使うよ（Pydanticと連携）

output_parser = PydanticOutputParser(pydantic_object=Recipe)  # Recipeという「レシピの型」に合わせて、AIの答えを分けてくれる準備をするよ


In [ ]:
format_instructions = output_parser.get_format_instructions()  # AIに「この形で答えてね！」って伝えるための説明文を取り出す
print(format_instructions)  # その説明文を見せるよ


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 会話のテンプレートを作るための道具を使うよ

prompt = ChatPromptTemplate.from_messages(  # 会話の流れを決めるテンプレートを作ってるよ
    [
        (  # 最初にAIにルールを教えるよ（システムメッセージ）
            "system",  # AIに伝える特別なメッセージだよ
            "ユーザーが入力した料理のレシピを考えてください。\n\n"  # AIにやってほしいこと（レシピを考える）を伝えるよ
            "{format_instructions}",  # あとでここに「こうやって出力してね」というルールが入るよ
        ),
        ("human", "{dish}"),  # 人間が料理の名前を入れる場所だよ（例：「カレー」）
    ]
)

prompt_with_format_instructions = prompt.partial(  # 上で作ったテンプレートの「穴」をうめていくよ
    format_instructions=format_instructions  # どういう形（JSON形式）で答えてほしいかのルールをここにセットするよ
)


In [ ]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})  # 「カレー」のレシピを考えてもらうようにAIにお願いするよ

print("=== role: system ===")  # AIにルールを伝える部分の見出しを表示するよ
print(prompt_value.messages[0].content)  # AIに「どうやって答えるか」のルールが書かれた文章を表示するよ

print("=== role: user ===")  # 人間からの質問部分の見出しを表示するよ
print(prompt_value.messages[1].content)  # 実際に人間が入力した「カレー」という料理名を表示するよ


=== role: system ===
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
カレー


In [ ]:
from langchain_openai import ChatOpenAI  # OpenAI（AIにおしゃべりしてもらう道具）を使えるようにするよ

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # gpt-4o-miniっていうAIを使って、答えがブレないようにしてるよ（temperature=0）

ai_message = model.invoke(prompt_value)  # AIに「これ考えて！」ってお願いするよ（prompt_valueの中身がそのお願い）

print(ai_message.content)  # AIが考えた答えを表示するよ（レシピの答えが見られるよ！）


{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、全体を混ぜる。",
    "水を加え、沸騰したらアクを取り、弱火で20分煮る。",
    "カレールーを加え、よく溶かしてさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}


In [ ]:
recipe = output_parser.invoke(ai_message)  # AIの返事（レシピの文章）を、ちゃんとした形（Recipeクラス）に変えるよ

print(type(recipe))  # 変換されたデータがどんな種類のものかを表示するよ（たとえば Recipe 型）

print(recipe)  # レシピの中身（材料や手順）を見せてくれるよ


<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、全体を混ぜる。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


### StrOutputParser


In [ ]:
from langchain_core.messages import AIMessage  # AIからの返事（メッセージ）を表すためのクラスだよ
from langchain_core.output_parsers import StrOutputParser  # AIの返事から文字を取り出す道具だよ

output_parser = StrOutputParser()  # パーサー（取り出す係）を用意するよ

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")  # AIが返してきたメッセージをつくるよ
ai_message = output_parser.invoke(ai_message)  # メッセージの中から文字（テキスト）だけを取り出すよ
print(type(ai_message))  # 結果がどんな種類のデータかを表示するよ（ここでは「文字列」になるよ）
print(ai_message)  # 実際の文字の中身を表示するよ（こんにちは〜って出てくるよ）


<class 'str'>
こんにちは。私はAIアシスタントです。


## 4.5.Chain―LangChain Expression Language（LCEL）の概要


### prompt と model の連鎖


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 🧠 プロンプト（AIにお願いする言葉）のテンプレートを使えるようにするよ
from langchain_openai import ChatOpenAI  # 🤖 OpenAIのAIモデルを使うための準備だよ

prompt = ChatPromptTemplate.from_messages(  # 📋 AIに渡す会話の流れを作ってるよ
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),  # 👩‍🏫「料理のレシピを考えてね」ってAIにルールを伝えてるよ
        ("human", "{dish}"),  # 👦 ユーザーが料理名を入れる場所だよ（たとえば「カレー」とか）
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)  # 🤖 「gpt-4o-mini」というAIを使うよ。温度0は「まじめに答えてね」って意味だよ


In [ ]:
chain = prompt | model  # 📦 プロンプト（お手紙の内容）とAI（読む人）をパイプでつなげて、1つの流れ（チェーン）を作ってるよ


In [ ]:
ai_message = chain.invoke({"dish": "カレー"})  # 🧾 「カレー」というお題をAIに送って、レシピをお願いしているよ！
print(ai_message.content)  # 📢 AIから返ってきた答え（レシピの内容）を画面に表示しているよ！


カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- 水: 800ml
- サラダ油: 大さじ2
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら火を止めます。

6. **味を調える**:
   - お好みでガーリックパウダーや生姜を加え、味を調整します。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、完成です。お好みで福神漬けやらっきょうを添えても良いでしょう。

### ポイント
- 野菜はお好みで追加しても良いです（ピーマン、ナス、ブロッコリーなど）。
- カレーは時間が経つほど味がなじむので、作り置きにも最適です。

美味しいカレーを楽しんでください！


### StrOutputParser を連鎖に追加


In [ ]:
from langchain_core.output_parsers import StrOutputParser  # 🧰 AIからの返事を「ただの文字」に変える道具を用意してるよ！

chain = prompt | model | StrOutputParser()  # 🧵 プロンプト → AI → 文字の形に変換、っていう流れをパイプでつなげてるよ！
output = chain.invoke({"dish": "カレー"})  # 📦 「カレーのレシピを作って！」ってAIにお願いして、答えをもらうよ！
print(output)  # 📢 その答え（レシピ）を画面に表示するよ！


カレーのレシピをご紹介します！シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら火を止めます。

6. **味を調える**:
   - お好みで塩や胡椒で味を調整します。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です！

### おすすめのトッピング
- チーズ
- 生卵（温泉卵や目玉焼き）
- 青ねぎやパセリの刻んだもの

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて楽しんでください！おいしいカレーをお楽しみください！


### PydanticOutputParser を使う連鎖


In [ ]:
from langchain_core.output_parsers import PydanticOutputParser  # 🧙‍♂️ AIの返事をキレイな形に整えてくれる道具を使うよ！
from pydantic import BaseModel, Field  # 📐 データの形（設計図）を作るための道具を読み込むよ！

class Recipe(BaseModel):  # 🍛 レシピの「型（かた）」を作るよ！
    ingredients: list[str] = Field(description="ingredients of the dish")  # 🥕 材料のリスト（たとえば「にんじん」「じゃがいも」など）だよ！
    steps: list[str] = Field(description="steps to make the dish")  # 🔪 作り方のステップ（たとえば「炒める」「煮る」など）を書くところだよ！

output_parser = PydanticOutputParser(pydantic_object=Recipe)  # 🧹 AIが出した答えをRecipeの形にピタッとはめてくれる道具を作るよ！


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 📦 AIに送るメッセージの形を作る道具
from langchain_openai import ChatOpenAI  # 🤖 OpenAIのチャットモデル（AI本体）を使うよ

# 📝 AIにお願いするメッセージを決めるよ（{dish}と{format_instructions}はあとで中身を入れる）
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。\n\n{format_instructions}"),  # 📋 システムメッセージで「レシピを考えてね」と伝える
        ("human", "{dish}"),  # 👤 人間が「カレー」みたいに料理名を入れるところ
    ]
)

# 🧾 AIに答えてほしい形式（JSON）を教えておくよ
prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()  # 📐 output_parserで自動生成した「出力ルール」を入れるよ
)

# 🤖 ChatGPTに「JSON形式で返してね」とお願いしておくよ
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}  # 🧊 「AIの返事はJSONだよ」ってルールを指定するよ
)


In [ ]:
chain = prompt_with_format_instructions | model | output_parser  # 🧩 プロンプト → AI → 出力整形をつなげて、1つの「命令チェーン」を作ってるよ


In [ ]:
recipe = chain.invoke({"dish": "カレー"})  # 🍛 「カレー」のレシピをAIにお願いして、結果をrecipeに入れる
print(type(recipe))  # 🧪 recipeの中身のデータの種類（型）を確認するよ（たとえば <class '__main__.Recipe'> って出る）
print(recipe)  # 📢 実際のレシピの内容を表示するよ


<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、沸騰したらアクを取り、弱火で20分煮る。', 'カレールーを加え、溶かしながらさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


### （コラム）with_structured_output


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 🤖 AIにどんな質問をするかの「型（テンプレート）」を使うよ
from langchain_openai import ChatOpenAI  # 🧠 OpenAI（ChatGPT）を使うための準備
from pydantic import BaseModel, Field  # 📦 レシピの「材料」と「手順」を入れる箱（クラス）を作るための道具

class Recipe(BaseModel):  # 🍽 レシピの形を定義するよ（材料と作り方を入れる）
    ingredients: list[str] = Field(description="ingredients of the dish")  # 📋 材料のリスト
    steps: list[str] = Field(description="steps to make the dish")  # 🧑‍🍳 作り方の手順リスト

prompt = ChatPromptTemplate.from_messages(  # 🗣 会話の流れを作るよ（誰が何を言うか）
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),  # 📢 AIへのルール説明
        ("human", "{dish}"),  # 🙋‍♂️ ユーザーが料理名（カレーなど）を入力するところ
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")  # 🧠 軽くて速いAIモデルを使うよ（GPT-4o mini）

chain = prompt | model.with_structured_output(Recipe)  # 🔗 入力→AI→レシピ形式に変換までをつなげたよ！

recipe = chain.invoke({"dish": "カレー"})  # 🍛 「カレー」のレシピを作ってもらう
print(type(recipe))  # 📦 レシピのデータの型（ちゃんとRecipe型になってるか）を確認
print(recipe)  # 📢 レシピの中身を表示（材料と手順）


<class '__main__.Recipe'>
ingredients=['鶏肉', '玉ねぎ', 'じゃがいも', 'にんじん', 'カレールー', '水', '塩', 'こしょう', 'サラダ油'] steps=['鶏肉は一口大に切り、塩とこしょうで下味をつける。', '玉ねぎは薄切り、じゃがいもとにんじんは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'じゃがいもとにんじんを加えて軽く炒める。', '水を加え、煮立ったらアクを取り除く。', '蓋をして中火で約15分煮る。', 'カレールーを加え、よく溶かしてさらに10分煮る。', '味を見て必要なら塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


## 4.6.LangChain の RAG に関するコンポーネント


### Document loader


In [ ]:
!pip install langchain-community==0.3.0 GitPython==3.1.43

  Using cached langchain_core-0.3.66-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.65-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.64-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.63-py3-none-any.whl.metadata (5.8 kB)
Using cached langchain_core-0.3.63-py3-none-any.whl (438 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.0
    Uninstalling langchain-core-0.3.0:
      Successfully uninstalled langchain-core-0.3.0
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.0
    Uninstalling langchain-text-splitters-0.3.0:
      Successfully uninstalled langchain-text-splitters-0.3.0


In [ ]:
from langchain_community.document_loaders import GitLoader  # GitHubからファイルを読み込む道具を使うよ

def file_filter(file_path: str) -> bool:  # ファイルの名前をチェックする関数を作るよ
    return file_path.endswith(".mdx")  # 「.mdx」で終わるファイルだけOKにするよ

loader = GitLoader(  # GitHubからファイルを読み込むためのローダーを作るよ
    clone_url="https://github.com/langchain-ai/langchain",  # GitHubの場所を指定するよ
    repo_path="./langchain",  # ここにダウンロードして保存するよ
    branch="master",  # 「master」ブランチを使うよ（最新の情報がある場所）
    file_filter=file_filter,  # さっき作った「.mdxだけ選ぶ」ルールを使うよ
)

raw_docs = loader.load()  # ファイルを全部読み込むよ
print(len(raw_docs))  # 読み込んだファイルの数を表示するよ


418


### Document transformer


In [ ]:
!pip install langchain-text-splitters==0.3.0  # テキストを分ける道具をインストールするよ（バージョン0.3.0）


  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
Using cached langchain_text_splitters-0.3.0-py3-none-any.whl (25 kB)
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.8
    Uninstalling langchain-text-splitters-0.3.8:
      Successfully uninstalled langchain-text-splitters-0.3.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.25 requires langchain-text-splitters<1.0.0,>=0.3.8, but you have langchain-text-splitters 0.3.0 which is incompatible.


In [ ]:
from langchain_text_splitters import CharacterTextSplitter  # 文字で文章を分ける道具を使うよ

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  # 1000文字ずつ切るよ、重なりはなしだよ

docs = text_splitter.split_documents(raw_docs)  # さっき読み込んだファイルを、1000文字ずつに分けるよ

print(len(docs))  # 分けたかけらの数を表示するよ


1454


### Embedding model


In [ ]:
from langchain_openai import OpenAIEmbeddings  # OpenAIの埋め込みモデルを使うよ（言葉→数字）

# 言葉をベクトル（数字）に変換するためのモデルを選ぶよ
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 小さめモデルを使うよ（速くて軽い！）


In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"  # 調べたい質問を用意するよ

vector = embeddings.embed_query(query)  # 質問をベクトル（数字のかたまり）に変えるよ

print(len(vector))  # ベクトルの長さ（何個の数字か）を表示するよ
print(vector)  # 実際のベクトルの中身（数字たち）を表示するよ


1536
[0.01968870498239994, -0.007562828715890646, 0.029617559164762497, -0.026554401963949203, 0.049940019845962524, 0.025012260302901268, -0.014977781102061272, 0.02129422128200531, 0.025730518624186516, -0.021463222801685333, 0.009273971430957317, -0.010615423321723938, -0.017417743802070618, -0.006010124925523996, -0.011428743600845337, 0.06316440552473068, 0.033335596323013306, -0.00034394499380141497, -0.04495446756482124, 0.02638540044426918, 0.0320047065615654, 0.03532136604189873, -0.03819439932703972, 0.021864602342247963, 0.01868525706231594, -0.018769757822155952, -0.020269649103283882, 0.032237086445093155, -0.00830221176147461, -0.10114755481481552, -0.009332065470516682, -0.0574183464050293, -0.034243982285261154, 0.04677123576402664, -0.023934874683618546, 0.034835487604141235, 0.023343367502093315, 0.013308888301253319, -0.00799589604139328, -0.033758100122213364, 0.00023254245752468705, -0.021009031683206558, 0.021072406321763992, 0.013171575032174587, 0.00648016110062

### Vector store


In [ ]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.3 MB/s eta 0

In [ ]:
from langchain_chroma import Chroma  # 意味ベースで検索できるデータベース「Chroma」を使うよ

db = Chroma.from_documents(docs, embeddings)  # さっき分割した文章たちと、意味をベクトルに変える道具を使って、検索できるデータベースを作るよ


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
retriever = db.as_retriever()  # ベクトル検索できるように変換するよ。意味が近い文を探せる検索機になるよ


In [ ]:
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"  # 質問の内容だよ

context_docs = retriever.invoke(query)  # retrieverに質問して、似ている文書を探してもらうよ
print(f"len = {len(context_docs)}")  # 何件の文書が見つかったかを表示するよ

first_doc = context_docs[0]  # 一番最初に見つかった文書を取り出すよ
print(f"metadata = {first_doc.metadata}")  # 文書にくっついてる追加情報（出典など）を表示するよ
print(first_doc.page_content)  # 実際の中身のテキストを表示するよ


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


len = 4
metadata = {'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) service that automaticall

### LCEL を使った RAG の Chain の実装


In [ ]:
from langchain_core.prompts import ChatPromptTemplate  # 会話用のひな形を作る道具を使うよ
from langchain_openai import ChatOpenAI  # OpenAIのAIモデルを使うよ

# 質問に答えてもらうときのひな形（プロンプト）を作るよ
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

# gpt-4o-miniという軽量のAIを使う準備をするよ
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
from langchain_core.output_parsers import StrOutputParser  # AIの返事を文字として取り出す道具だよ
from langchain_core.runnables import RunnablePassthrough  # そのまま渡すためのパイプみたいなものだよ

# 「文脈を取ってきて、質問を渡して、AIの答えを文字にする」流れを作るよ
chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # retrieverが文脈を探して、質問をそのまま渡すよ
    | prompt  # テンプレートに当てはめるよ（「文脈：〜 質問：〜」って形にするよ）
    | model  # AIに答えてもらうよ
    | StrOutputParser()  # AIの答えを「ただの文字」にして取り出すよ
)

# 実際に質問して答えをもらうよ
output = chain.invoke(query)
print(output)  # AIが返してくれた答えを表示するよ


はい、AWSのS3からデータを読み込むためのDocument loaderとして、`S3DirectoryLoader`と`S3FileLoader`があります。これらは、AWS S3のディレクトリやファイルからデータを読み込むために使用されます。
